<font color = "#CC3D3D">
# EX: Apply XGBoost to ClickStream Data

### Load classes ###

In [1]:
# python basic classes
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# scikit-learn commonly used classes
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

# scikit-learn popular classifiers
from sklearn.ensemble import RandomForestClassifier

# XGBoost
from xgboost import XGBClassifier

### Read data ###

In [2]:
log = pd.read_csv("data_clickstreams.csv", encoding='cp949') 
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 40 columns):
CUS_ID            2000 non-null int64
CT_PORTAL         2000 non-null float64
CT_SEARCH         2000 non-null float64
CT_SHOP           2000 non-null float64
CT_NEWS           2000 non-null float64
CT_MAIL           2000 non-null float64
CT_COMMUNITY      2000 non-null float64
CT_BLOG_SNS       2000 non-null float64
CT_ENTERTAIN      2000 non-null float64
CT_FINANCE        2000 non-null float64
CT_SHOP_INFO      2000 non-null float64
CT_BUSINESS       2000 non-null float64
CT_INTERNET       2000 non-null float64
CT_DOWNLOAD       2000 non-null float64
CT_PUBLIC         2000 non-null float64
CT_GAME           2000 non-null float64
CT_EDU            2000 non-null float64
CT_ETC            2000 non-null float64
COVERAGE          2000 non-null float64
DWELLTIME         2000 non-null int64
PAGEVIEWS         2000 non-null int64
HF0005            2000 non-null float64
HF0611            2

<font color = "blue">
### Feature Description: ###
<p>
- **CUS_ID**: 고객 ID (총 2,000명)
- **CT_xxx**: 웹사이트 카테고리 별 체류시간 비율. 즉, 총 17개 카테고리 중에 특정 카테고리에 얼마나 머물렀는가를 비율로 계산한 값
- **COVERAGE**: 서로 다른 웹사이트에 얼마나 다양하게 접속했는지에 대한 비율("서로 다른 카테고리 수/17"로 계산)
- **DWELLTIME**: 총 체류시간
- **PAGEVIEWS**: 총 페이지뷰
- **HF_xxx**: 시간대별(0-5시, 6-11시, 12-17시, 18-23시) 체류시간 비율
- **DF_xxx**: 요일별 체류시간 비율
- **VSITES**: 접속한 서로 다른 웹사이트의 수
- **SITECOV**: 웹사이트 카테고리 별 체류시간 변동계수(카테고리별 체류시간의 "표준편차/평균" 값)
- **VDAYS**: 총 접속 일수 
- **DAYTIME**: 하루 평균 체류시간 
- **DAYCOV**: 일별 변동계수(일일 체류시간의 "표준편차/평균" 값) 
- **SCH_KEYWORDS**: 네이버에서 검색한 검색량
- **SCH_TOPKEYWORD**: 네이버에서 가장 많이 검색한 검색어
- **GENDER**: 고객 성별(남자/여자). 예측하고자 하는 값.

In [3]:
log.head()

,CUS_ID,CT_PORTAL,CT_SEARCH,CT_SHOP,CT_NEWS,CT_MAIL,CT_COMMUNITY,CT_BLOG_SNS,CT_ENTERTAIN,CT_FINANCE,...,DF_SAT,DF_SUN,VSITES,SITECOV,VDAYS,DAYTIME,DAYCOV,SCH_KEYWORDS,SCH_TOPKEYWORD,GENDER
0,1,0.093,0.136,0.001,0.280,0.029,0.192,0.029,0.205,0.001,...,0.081,0.129,201,4.272,205,1746,1.481,399,타이젬,남자
1,2,0.059,0.118,0.323,0.028,0.119,0.017,0.072,0.015,0.011,...,0.007,0.000,573,5.434,105,7962,0.484,1346,티몬,여자
2,3,0.430,0.107,0.179,0.019,0.164,0.012,0.017,0.007,0.001,...,0.026,0.038,270,5.965,105,3699,0.700,329,야놀자,여자
3,4,0.218,0.078,0.039,0.260,0.041,0.173,0.066,0.004,0.037,...,0.054,0.037,166,4.309,107,2575,1.144,134,아이엠간지,남자
4,5,0.332,0.223,0.033,0.200,0.003,0.039,0.039,0.003,0.046,...,0.132,0.028,328,5.750,151,3324,1.058,511,에스티아이,남자


### Encode categorical values using one-hot encoding ###

In [4]:
encoded_log = pd.get_dummies(log, columns=['SCH_TOPKEYWORD'])

### Split data ###

In [5]:
encoded_log.shape

(2000, 1496)

In [6]:
dfX = encoded_log.drop(['CUS_ID','GENDER'], axis=1) # exclude 'CUS_ID' attribute & class variable
dfy = encoded_log['GENDER']                         # class variable
X_train, X_test, y_train, y_test = train_test_split(dfX, dfy, test_size=0.2, random_state=0)

### Tune models ###

##### Set hyper-parameters #####

In [7]:
rf_params = {
    'rf__max_features': np.arange(5,10),
    'rf__n_estimators': [100,300,500]
}

xgb_params = {
    'xgb__subsample': np.arange(.5, 1.0, .2),
    'xgb__max_depth': np.arange(3,10,3),
#    'xgb__colsample_bytree': np.arange(.1,1.05,.05)
}

##### Make pipeline #####

In [8]:
rf_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("rf", RandomForestClassifier(random_state=0, n_jobs=-1))
])

xgb_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("xgb", XGBClassifier(tree_method = 'hist', random_state=0, n_jobs=-1))
])

In [9]:
models = [
    ('rf', rf_pipeline, rf_params),
    ('xgb', xgb_pipeline, xgb_params),
]

##### Run grid search & CV #####

In [10]:
%%time
tuned_models = {}
best_score = -1
best_model = None
for name, model, param in models :
    grid_search = GridSearchCV(model, param, cv=5).fit(X_train, y_train)
    score = grid_search.score(X_test, y_test)
    print("{} ==> {} {}".format(name, score, grid_search.best_params_))
    tuned_models[name] = grid_search
    if score > best_score : 
        best_score = score
        best_model = name

rf ==> 0.6475 {'rf__max_features': 7, 'rf__n_estimators': 300}
xgb ==> 0.6725 {'xgb__max_depth': 9, 'xgb__subsample': 0.8999999999999999}
CPU times: user 22h 47min 5s, sys: 12min 31s, total: 22h 59min 37s
Wall time: 42min 50s


<font color = "#CC3D3D">
# End